In [7]:
!pip install gradio

In [8]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr

In [12]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    features = sms_text_df['text_message'] # Set features to the text message column
    
    # Set the target variable to the "label" column.
    target = sms_text_df['label'] # Set target to the label column
    
    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42) # Split the data
    
 # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LinearSVC()) 
    ])
    
    # Fit the model to the transformed training data and return model.
    text_clf.fit(X_train, y_train) # Fit the model to the training data
    return text_clf

In [13]:
#Load dataset into a DataFrame
sms_text_df = pd.read_csv("Resources/SMSSpamCollection.csv", sep=',')
# Display the first few rows of the DataFrame to verify the data is loaded correctly
sms_text_df.head() # Display the first few rows of dataframe



,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [14]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_text_df)

In [15]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    prediction = text_clf.predict([text])[0]
    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.' # Return this message if the prediction is "ham"
    else:
        return f'The text message: "{text}", is spam.' # Return this message if the prediction is "spam"

    

In [16]:
# Create a Gradio Interface application for SMS classification
def sms_prediction_interface():
    """
    Create and launch a Gradio Interface application for SMS spam detection.

    This function creates a Gradio Interface application that takes a text input for SMS messages
    and provides a prediction of whether the message is spam or not.
    """
    # Define the input and output components for the Gradio Interface
    input_textbox = gr.Textbox(lines=2, placeholder="Enter your SMS text here...", label="Input SMS Text")
    output_textbox = gr.Textbox(label="Prediction")

    # Create the Gradio Interface application with the defined inputs, outputs, and the prediction function
    sms_app = gr.Interface(fn=sms_prediction, inputs=input_textbox, outputs=output_textbox, 
                           title="SMS Spam Detection", description="Enter an SMS text to check if it is spam or not.")

    # Launch the app with a public link
    sms_app.launch(share=True)

# Call function to create and launch the Gradio Interface application
sms_prediction_interface()

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: C:\Users\gasho\anaconda3\Lib\site-packages\gradio\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\gasho\anaconda3\Lib\site-packages\gradio


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.

In [ ]:
Answer
The text message: "You are a lucky winner of $5000!", is not spam
The text message: "You won 2 free tickets to the Super Bowl", is spam.
The text message: "You won 2 free tickets to the Super Bowl text us to claim your prize.", is spam.
The text message: "You won 2 free tickets to the Super Bowl text us to claim your prize.", is spam.
